<a href="https://colab.research.google.com/github/sgzmd/leetformer/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from tqdm import tqdm

In [2]:
# Load dataset
df = pd.read_csv("data/1000_leetspeak.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Original   9999 non-null   object
 1   LeetSpeak  9999 non-null   object
dtypes: object(2)
memory usage: 156.4+ KB


In [3]:
# Initialize tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# change to cuda for GPU acceleration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [4]:
# torch.backends.cuda.is_available()

In [5]:
class LeetSpeakDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=32):
        self.data = dataframe.to_dict(orient='records')  # Convert to list of dicts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        original = str(sample['Original'])
        leetspeak = str(sample['LeetSpeak'])

        source = self.tokenizer(
            "convert: " + original,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        target = self.tokenizer(
            leetspeak,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            "input_ids": source["input_ids"].squeeze(),
            "attention_mask": source["attention_mask"].squeeze(),
            "labels": target["input_ids"].squeeze()
        }

In [6]:
train_size = int(0.8 * len(df))
test_size = len(df) - train_size
indices = list(range(len(df)))
train_indices, test_indices = indices[:train_size], indices[train_size:]

full_dataset = LeetSpeakDataset(df, tokenizer)
train_dataset = Subset(full_dataset, train_indices)
test_dataset = Subset(full_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
EPOCHS = 5
model.train()
for epoch in range(EPOCHS):
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

# Save model
model.save_pretrained("leetspeak_model")
tokenizer.save_pretrained("leetspeak_model")


Epoch 5: 100%|██████████| 1000/1000 [01:17<00:00, 12.96it/s, loss=0.4]


('leetspeak_model/tokenizer_config.json',
 'leetspeak_model/special_tokens_map.json',
 'leetspeak_model/spiece.model',
 'leetspeak_model/added_tokens.json')

In [9]:
def generate_leetspeak(text):
    model.eval()
    input_text = "convert: " + text
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

    with torch.no_grad():
        output_ids = model.generate(input_ids)

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Example inference
print(generate_leetspeak("good morning"))

b0rm3nt


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Load dataset
df = pd.read_csv("data/1000_leetspeak.csv")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom PyTorch Dataset
class LeetSpeakDataset(Dataset):
    def __init__(self, dataframe, max_length=32):
        self.data = dataframe.to_dict(orient='records')
        self.max_length = max_length
        self.char_set = set("".join(data['Original'] + data['LeetSpeak'] for data in self.data))
        self.char_to_idx = {char: idx for idx, char in enumerate(sorted(self.char_set), start=1)}
        self.idx_to_char = {idx: char for char, idx in self.char_to_idx.items()}

    def __len__(self):
        return len(self.data)

    def encode(self, text):
        return [self.char_to_idx[char] for char in text] + [0] * (self.max_length - len(text))

    def __getitem__(self, idx):
        sample = self.data[idx]
        original = self.encode(sample['Original'])
        leetspeak = self.encode(sample['LeetSpeak'])
        return torch.tensor(original, dtype=torch.long), torch.tensor(leetspeak, dtype=torch.long)

# Define LSTM Model
class LeetSpeakLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_dim=128, output_dim=None):
        super(LeetSpeakLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size if output_dim is None else output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        output = self.fc(lstm_out)
        return output

# Prepare dataset
train_size = int(0.8 * len(df))
test_size = len(df) - train_size
train_df, test_df = df.iloc[:train_size], df.iloc[train_size:]

train_dataset = LeetSpeakDataset(train_df)
test_dataset = LeetSpeakDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Initialize model
vocab_size = len(train_dataset.char_to_idx) + 1
model = LeetSpeakLSTM(vocab_size).to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
EPOCHS = 5
model.train()
for epoch in range(EPOCHS):
    loop = tqdm(train_loader, leave=True)
    for original, leetspeak in loop:
        optimizer.zero_grad()
        original, leetspeak = original.to(device), leetspeak.to(device)
        output = model(original)
        loss = criterion(output.view(-1, vocab_size), leetspeak.view(-1))
        loss.backward()
        optimizer.step()
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

# Save model
torch.save(model.state_dict(), "leetspeak_lstm.pth")

# Function to generate leetspeak
def generate_leetspeak(model, dataset, text):
    model.eval()
    encoded = torch.tensor([dataset.encode(text)], dtype=torch.long).to(device)
    with torch.no_grad():
        output = model(encoded)
    decoded = [dataset.idx_to_char[idx] for idx in output.argmax(-1).squeeze().tolist() if idx != 0]
    return "".join(decoded)

# Example inference
print(generate_leetspeak(model, train_dataset, "hello"))
